Movie review Dataset: https://www.kaggle.com/pankrzysiu/keras-imdb <br>
Global vectors for word representation: https://www.kaggle.com/rtatman/glove-global-vectors-for-word-representation        

NLP and Word Embeddings


> Natural language processing (NLP) is a field of computer science, artificial intelligence concerned with the interactions between computers and human (natural) languages, and, in particular, concerned with programming computers to fruitfully process large natural language data.

> Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural language generation.


## Sentiment analysis with the IMDB dataset

Sentiment analysis is about judging how positive or negative the tone in a document is. The output of a sentiment analysis is a score between zero and one, where one means the tone is very positive and zero means it is very negative. Sentiment analysis is used for trading quite frequently. For example the sentiment of quarterly reports issued by firms is automatically analyzed to see how the firm judges its own position. Sentiment analysis is also applied to the tweets of traders to estimate an overall market mood. Today, there are many data providers that offer sentiment analysis as a service.

In principle, training a sentiment analysis model works just like training a binary text classifier. The text gets classified into positive (1) or not positive (0). This works exactly like other binary classification only that we need some new tools to handle text.

A common dataset for sentiment analysis is the corpus of [Internet Movie Database (IMDB)](http://www.imdb.com/) movie reviews. Since each review comes with a text and a numerical rating, the number of stars, it is easy to label the training data. In the IMDB dataset, movie reviews that gave less then five stars where labeled negative while movies that gave more than seven stars where labeled positive (IMDB works with a ten star scale). Let's give the data a look:

In [1]:
from tqdm import tqdm

In [3]:
import os

imdb_dir = 'aclImdb_v1/aclImdb' # Data directory
train_dir = os.path.join(imdb_dir, 'train') # Get the path of the train set

# Setup empty lists to fill
labels = []
texts = []

# First go through the negatives, then through the positives
for label_type in ['neg', 'pos']:
    # Get the sub path
    dir_name = os.path.join(train_dir, label_type)
    print('loading ',label_type)
    # Loop over all files in path
    for fname in tqdm(os.listdir(dir_name)):
        
        # Only consider text files
        if fname[-4:] == '.txt':
            # Read the text file and put it in the list
            f = open(os.path.join(dir_name, fname), 'rb')
            texts.append(f.read())
            f.close()
            # Attach the corresponding label
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

loading  neg


100%|██████████| 12500/12500 [01:11<00:00, 173.86it/s]


loading  pos


100%|██████████| 12500/12500 [01:24<00:00, 148.50it/s]


We should have 25,000 texts and labels.

In [4]:
len(labels), len(texts)

(25000, 25000)

Half of the reviews are positive

In [5]:
import numpy as np
np.mean(labels)

0.5

Let's look at a positive review:

In [6]:
print('Label',labels[24002])
print(texts[24002])

Label 1
b"I saw this movie at the 18th Haifa film festival, and it is one of the best I've seen this year. Seeing it on a big screen (and I mean BIG, not one of those TV screens most cinemas have) with an excellent sound system always enhance the cinematic experience, as the movie takes over your eyes and ears and sucks you into the story, into the picture.<br /><br />The movie presents a set of characters, which are loosely inter-connected. Their stories cross at certain points, and the multiplicity of story lines reminded me very much of the great Robert Altman and his exquisite films. But the true hero of the movie is obviously the city of Madrid, which provides the backdrop for the entire movie. It houses the characters, contains the pavements and roads on which they walk, and sets the background atmosphere for all the events, all in beautifully filmed scenes.<br /><br />The movie returns again and again to certain themes (shoes, for instance), and in essence Salazar makes his meta

And a negative review:

In [7]:
print('Label',labels[1])
print(texts[1])

Label 0
b"Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to his estate in preparation of it being opened to the public as a museum, also on board is Stevens daughter Julie (Kathleen Quinlan) & her son. The luxury jetliner takes off as planned but mid-air the plane is hi-jacked by the co-pilot Chambers (Robert Foxworth) & his two accomplice's Banker (Monte Markham) & Wilson (Michael Pataki) who knock the passengers & crew out with sleeping gas, they plan to steal the valuable cargo & land on a disused plane strip on an isolated island but while making his descent Chambers almost hits an oil rig in the Ocean & loses control of the plane sending it crashing into the sea where it sinks to the bottom right bang in the middle of the Bermuda Triangle. With air in short supply, water leaking in & having flown over 200 miles off course the problems m

## Tokenizing text

Computers can not work with words directly. To them, a word is just a meaningless row of characters. To work with words, we need to turn words into so called 'Tokens'. A token is a number that represents that word. Each word gets assigned a token. Tokens are usually assigned by word frequency. The most frequent words like 'a' or 'the' get tokens like 1 or 2 while less often used words like 'profusely' get assigned very high numbers.

We can tokenize text directly with Keras. When we tokenize text, we usually choose a maximum number of words we want to consider, our vocabulary so to speak. This prevents us from assigning tokens to words that are hardly ever used, mostly because of typos or because they are not actual words or because they are just very uncommon. This prevents us from over fitting to texts that contain strange words or wired spelling errors. Words that are beyond that cutoff point get assigned the token 0, unknown.

In [14]:
type(texts)

list

In [17]:
from keras.preprocessing.text import Tokenizer
import numpy as np

max_words = 10000 # We will only consider the 10K most used words in this dataset

tokenizer = Tokenizer(num_words=max_words) 
tokenizer.fit_on_texts([x.decode('utf-8') for x in texts]) # Generate tokens by counting frequency
sequences = tokenizer.texts_to_sequences([x.decode('utf-8') for x in texts]) # Turn text into sequence of numbers

The tokenizers word index is a dictionary that maps each word to a number. You can see that words that are frequently used in discussions about movies have a lower token number.

In [18]:
word_index = tokenizer.word_index
print('Token for "the"',word_index['the'])
print('Token for "Movie"',word_index['movie'])
print('Token for "generator"',word_index['generator'])

Token for "the" 1
Token for "Movie" 17
Token for "generator" 20385


Our positive review from earlier has now been converted into a sequence of numbers.

In [19]:
# Display the first 10 words of the sequence tokenized
sequences[24002][:10]

[10, 216, 11, 17, 30, 1, 9410, 19, 1413, 2]

To proceed, we now have to make sure that all text sequences we feed into the model have the same length. We can do this with Keras pad sequences tool. It cuts of sequences that are too long and adds zeros to sequences that are too short.

In [21]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100 # Make all sequences 100 words long
data = pad_sequences(sequences, maxlen=maxlen)
print(data.shape) # We have 25K, 100 word sequences now

(25000, 100)


Now we can turn all data into proper training and validation data.

In [23]:
labels = np.asarray(labels)

# Shuffle data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

training_samples = 20000  # We will be training on 10K samples
validation_samples = 5000  # We will be validating on 10000 samples



In [24]:
# Split data
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

## Embeddings

words and word tokens are categorical features. As such, we can not directly feed them into the neural net. Just because a word has a larger token value, it does not express a higher value in any way. It is just a different category. Previously, we have dealt with categorical data by turning it into one hot encoded vectors. But for words, this is impractical. Since our vocabulary is 10,000 words, each vector would contain 10,000 numbers which are all zeros except for one. This is highly inefficient. Instead we will use an embedding. 

Embeddings also turn categorical data into vectors. But instead of creating a one hot vector, we create a vector in which all elements are numbers.

![Embedding](https://storage.googleapis.com/aibootcamp/Week%204/assets/embeddings.png)

In practice, embeddings work like a look up table. For each token, they store a vector. When the token is given to the embedding layer, it returns the vector for that token and passes it through the neural network. As the network trains, the embeddings get optimized as well. Remember that neural networks work by calculating the derivative of the loss function with respect to the parameters (weights) of the model. Through backpropagation we can also calculate the derivative of the loss function with respect to the _input_ of the model. Thus we can optimize the embeddings to deliver ideal inputs that help our model. 

In practice it looks like this: We have to specify how large we want the word vectors to be. A 50 dimensional vector is able to capture good embeddings even for quite large vocabularies. We also have to specify for how many words we want embeddings and how long our sequences are.

In [45]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

embedding_dim = 50

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 50)           500000    
                                                                 
 flatten_2 (Flatten)         (None, 5000)              0         
                                                                 
 dense_4 (Dense)             (None, 32)                160032    
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 660,065
Trainable params: 660,065
Non-trainable params: 0
_________________________________________________________________


You can see that the embedding layer has 500,000 trainable parameters, that is 50 parameters for each of the 10K words.

In [46]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [47]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Epoch 1/10
625/625 [==============================] - 6s 9ms/step - loss: 0.4341 - acc: 0.7884 - val_loss: 0.3431 - val_acc: 0.8552
Epoch 2/10
625/625 [==============================] - 5s 9ms/step - loss: 0.1161 - acc: 0.9611 - val_loss: 0.4865 - val_acc: 0.8218
Epoch 3/10
625/625 [==============================] - 5s 9ms/step - loss: 0.0109 - acc: 0.9983 - val_loss: 0.5617 - val_acc: 0.8362
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.5945 - val_acc: 0.8406
Epoch 5/10
625/625 [==============================] - 5s 9ms/step - loss: 4.5886e-04 - acc: 1.0000 - val_loss: 0.6259 - val_acc: 0.8412
Epoch 6/10
625/625 [==============================] - 5s 9ms/step - loss: 2.5353e-04 - acc: 1.0000 - val_loss: 0.6558 - val_acc: 0.8428
Epoch 7/10
625/625 [==============================] - 5s 9ms/step - loss: 1.5685e-04 - acc: 1.0000 - val_loss: 0.6787 - val_acc: 0.8432
Epoch 8/10
625/625 [==============================] - 5s 8ms/ste

In [48]:
glove_dir = '' # This is the folder with the dataset

print('Loading word vectors')
embeddings_index = {} # We create a dictionary of word -> embedding
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), 'rb') # Open file

# In the dataset, each line represents a new word embedding
# The line starts with the word and the embedding values follow

for line in tqdm(f):
    values = line.split()
    word = values[0] # The first value is the word, the rest are the values of the embedding
    embedding = np.asarray(values[1:], dtype='float32') # Load embedding
    embeddings_index[word] = embedding # Add embedding to our embedding dictionary
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Loading word vectors


400000it [00:15, 26351.54it/s]

Found 400000 word vectors.


Not all words that are in our IMDB vocabulary might be in the GloVe embeddings though. For missing words it is wise to use random embeddings with the same mean and standard deviation as the GloVe embeddings

In [49]:
# Create a matrix of all embeddings
all_embs = np.stack(embeddings_index.values())
emb_mean = all_embs.mean() # Calculate mean
emb_std = all_embs.std() # Calculate standard deviation
emb_mean,emb_std

(0.004451992, 0.4081574)

We can now create an embedding matrix holding all word vectors.

In [50]:
embedding_dim = 100 # We now use larger embeddings

word_index = tokenizer.word_index
nb_words = min(max_words, len(word_index)) # How many words are there actually

# Create a random matrix with the same mean and std as the embeddings
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_dim))

# The vectors need to be in the same position as their index. 
# Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on

# Loop over all words in the word index
for word, i in word_index.items():
    # If we are above the amount of words we want to use we do nothing
    if i >= max_words: 
        continue
    # Get the embedding vector for the word
    embedding_vector = embeddings_index.get(word)
    # If there is an embedding vector, put it in the embedding matrix
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

This embedding matrix can be used as weights for the embedding layer. This way, the embedding layer uses the pre trained GloVe weights instead of random ones. We can also set the embedding layer to not trainable. This means, Keras won't change the weights of the embeddings while training which makes sense since our embeddings are already trained.

In [51]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, weights = [embedding_matrix], trainable = False))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 flatten_3 (Flatten)         (None, 10000)             0         
                                                                 
 dense_6 (Dense)             (None, 32)                320032    
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,320,065
Trainable params: 320,065
Non-trainable params: 1,000,000
_________________________________________________________________


Notice that we now have far fewer trainable parameters.

In [52]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [53]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 0.6702 - acc: 0.5903 - val_loss: 0.6627 - val_acc: 0.6032
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4302 - acc: 0.8030 - val_loss: 0.7422 - val_acc: 0.6034
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1676 - acc: 0.9385 - val_loss: 0.9975 - val_acc: 0.6086
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0583 - acc: 0.9801 - val_loss: 1.2534 - val_acc: 0.6006
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0256 - acc: 0.9912 - val_loss: 1.4413 - val_acc: 0.6044
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0139 - acc: 0.9948 - val_loss: 1.6062 - val_acc: 0.6034
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.0113 - acc: 0.9951 - val_loss: 1.8055 - val_acc: 0.6030
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.

Now our model over fits less but also does worse on the validation set.

# Using our model

To determine the sentiment of a text, we can now use our trained model.

In [54]:
# Demo on a positive text
my_text = 'I love dogs. Dogs are the best. They are lovely, cuddly animals that only want the best for humans.'

seq = tokenizer.texts_to_sequences([my_text])
print('raw seq:',seq)
seq = pad_sequences(seq, maxlen=maxlen)
print('padded seq:',seq)
prediction = model.predict(seq)
print('positivity:',prediction)

raw seq: [[10, 116, 2517, 2517, 23, 1, 115, 33, 23, 1333, 1388, 12, 61, 178, 1, 115, 15, 1708]]
padded seq: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   10  116
  2517 2517   23    1  115   33   23 1333 1388   12   61  178    1  115
    15 1708]]
positivity: [[0.9561363]]


In [55]:
# Demo on a negative text
my_text2 = 'The bleak economic outlook will force many small businesses into bankruptcy.'

seq = tokenizer.texts_to_sequences([my_text2])
print('raw seq:',seq)
seq = pad_sequences(seq, maxlen=maxlen)
print('padded seq:',seq)
prediction = model.predict(seq)
print('positivity:',prediction)

raw seq: [[1, 3741, 7050, 77, 1145, 108, 389, 80]]
padded seq: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    1 3741 7050   77 1145  108
   389   80]]
positivity: [[0.01941606]]


## Word embeddings as semantic geometry

One very interesting aspect of embeddings trained on large numbers of words is that they show patterns in which the geometric relationship between word vectors corresponds to the semantic relationship between these words.

![Relations](https://storage.googleapis.com/aibootcamp/Week%204/assets/man_woman.jpg)


In the picture above for instance you can see that the direction of feminine words to their male counterparts is roughly the same. In other words, if you where to substract the word vector for 'woman' from the word 'queen' and add the word vector for 'man' you would arrive at 'king'. This also works for other relationships like comparatives and superlatives. 

![Rel Comp Sup](https://storage.googleapis.com/aibootcamp/Week%204/assets/comparative_superlative.jpg)

This highlights some interesting properties of language in which semantic meanings can be seen as directions which can be added or subtracted.

A sad side effect of training word vectors on human writing is that it captures human biases. For example it has been [shown](https://www.technologyreview.com/s/602025/how-vector-space-mathematics-reveals-the-hidden-sexism-in-language/) that for word vectors trained on news websites, 'Programmer' - 'Man' + 'Woman' equals 'Homemaker' reflecting the bias in language that assigns the role of homemaker more often to woman than men. Measuring these biases in embeddings and correcting them has become a field of [research on its own](https://www.technologyreview.com/s/602025/how-vector-space-mathematics-reveals-the-hidden-sexism-in-language/) which highlights how even professional writing from news outlets can be biased.